# Experimenting with Parallel Teleport on Neutral Atom Device

This notebook shows how you can use the `NeutralAtomDevice` class to visualize and simulate a scaled, parallel teleport sample program.

First, set up the environment.

In [ ]:
from qdk import init, TargetProfile, code, Result
from qdk.qsharp import compile, circuit
from qdk.simulation import NeutralAtomDevice, NoiseConfig
from qdk.widgets import Circuit, Histogram

init(target_profile=TargetProfile.Base)

Next, we use Q# to define a `ParalellTeleport` that uses distinct sets of qubits to perform a measurement deferred teleportation. Using different qubits rather than the same qubits should allow later device-specific compilation steps to identify parallelism and run those teleports across the qubits at the same time.

In [ ]:
%%qsharp

operation ParallelTeleport(instances : Int) : Result[] {
    // Partitions the teleport instances across rows and columns on the machine
    let cols = if instances >= 12 { 36 } else { instances * 3 };
    let rows = (instances + 11) / 12;  // 1 to 12 = 1, 13 to 24 = 2, etc.

    use qubits = Qubit[instances * 3];
    mutable results : Result[] = [];

    for i in 0..instances-1 {
        let rowId = i / 12;
        let colId = (i % 12) * 3;
        let idx = colId + (rowId * 36);

        // Prep state on msg qubit
        if i % 4 == 1 {
            X(qubits[idx]);
        } elif i % 4 == 2 {
            H(qubits[idx]);
        } elif i % 4 == 3 {
            SX(qubits[idx]);
        }

        Teleport(qubits[idx], qubits[idx + 1], qubits[idx + 2]);

        // Reverse state prep on Bob's qubit
        if i % 4 == 1 {
            X(qubits[idx + 2]);
        } elif i % 4 == 2 {
            H(qubits[idx + 2]);
        } elif i % 4 == 3 {
            X(qubits[idx + 2]);
            SX(qubits[idx + 2]);
        }
        results += [MResetZ(qubits[idx + 2])];
        ResetAll([qubits[idx], qubits[idx + 1]]);
    }

    return results;
}

operation Teleport(msg : Qubit, alice : Qubit, bob : Qubit) : Unit {
    // Create some entanglement that we can use to send our message.
    H(alice);
    CNOT(alice, bob);

    // Encode the message into the entangled pair.
    CNOT(msg, alice);
    H(msg);

    CNOT(alice, bob);
    Controlled Z([msg], bob);
}

You can verify the circuit for two parallel teleports using the `Circuit` widget.

In [ ]:
Circuit(circuit(code.ParallelTeleport, 2))

Now, let's visualize 120 instances of teleportation in parallel on the `NeutralAtomDevice`. This will decompose the program to the device's native gate set and schedule the operations with movement, showing how it might execute on such a device.

In [ ]:
# Compile the code into a high-level program
program = compile(code.ParallelTeleport, 120)

# Visualize a trace of running the program on the target machine,
# decomposing into native gates and scheduling parallel operations
device = NeutralAtomDevice()
device.show_trace(program)

Because the teleport algorithm uses only gates from the Clifford group, we can efficiently simulate a large number of shots of the same program with the Clifford simulator. We'll use a small helper function to interpret the results, which we expect to be all zeros. Without noise, "Correct" should be the only bar in the resulting Histogram.

In [ ]:
results = device.simulate(program, shots=1000, type="clifford")
Histogram(["Loss" if any(r == Result.Loss for r in shot) else "Flip" if any(r == Result.One for r in shot) else "Correct" for shot in results])

You can also perform this simulation with noise, shown here by configuring a 0.1% chance of loss on the SX gate. Now we'll see a mix of correct and incorrect results in the output of the simulation.

In [ ]:
# Now with noise.
noise = NoiseConfig()
noise.sx.loss = 0.001
results = device.simulate(program, shots=1000, noise=noise, type="clifford")
Histogram(["Loss" if any(r == Result.Loss for r in shot) else "Flip" if any(r == Result.One for r in shot) else "Correct" for shot in results])